2nd Model training of the data with out hyper tunning.  
In this Model training we are not removing any outliers.  
Only th columns transformed to new columns will be removed. 

### In this model i have used OneHotEncoding.  
The techniques used for model training are as below with out any hpyer tunning.  
1. Linear regression.  
2. Lasso.  
3. Elastic net.  
4. Random forest.  
5. Decission Tree.  
6. Logistic Regression.  
7. LinearSVC.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import os
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
## Model Training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.svm import LinearSVC

In [2]:
path=os.path.join("Data","flight_dataset.csv")
path

'Data/flight_dataset.csv'

In [3]:
data=pd.read_csv(path)
df=data.copy()
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [4]:
def convert_date_format(Data,Column):
    """
    This function is used to convert the given date format to a correct format.
    Complete date column will be splited into saperate columns
    Date, Month, Year this are the new columns to be created.
    """ 

    Data[Column]=pd.to_datetime(Data[Column],infer_datetime_format=True)
    Data['Day']=Data[Column].dt.day
    Data['Month']=Data[Column].dt.month
    Data['Year']=Data[Column].dt.year

In [5]:
convert_date_format(df,"Date_of_Journey")

/tmp/ipykernel_470/3225999564.py:8: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  Data[Column]=pd.to_datetime(Data[Column],infer_datetime_format=True)
/tmp/ipykernel_470/3225999564.py:8: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  Data[Column]=pd.to_datetime(Data[Column],infer_datetime_format=True)


In [6]:
def convert_time_format(Data,deptime,arivaltime):
    
    """
    This function convert th 24:00 Hr time format to 2 saperate index hour and min.
    New columns with Departure Hour, Arival Hour, Departure Min and Arival Min will be created.
    This function can be used when the time is in 00:00 this format. other format will not work(ex 00.00)
    """ 
    
    Data["Dep_hour"]=Data[deptime].str.split(":").str[0]
    Data["Dep_min"]=Data[deptime].str.split(":").str[1]

    Data[arivaltime]=Data[arivaltime].str.split(" ").str[0]
    Data["Arival_hour"]=Data[arivaltime].str.split(":").str[0]
    Data["Arival_min"]=Data[arivaltime].str.split(":").str[1]

In [7]:
convert_time_format(df,"Dep_Time","Arrival_Time") 

In [8]:
df[df["Duration"] == "5m"]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Day,Month,Year,Dep_hour,Dep_min,Arival_hour,Arival_min
6474,Air India,2019-03-06,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327,6,3,2019,16,50,16,55


In [9]:
df.drop(6474,axis=0,inplace=True)

In [10]:
def convert_travel_time(Data,DurationTime):
    """
    This function is used to conver the travelling time to saperate columns.
    Function will create 2 columns travelling hour and travelling min.
    """ 

    Data[DurationTime]=Data[DurationTime].str.split(" ")

    Data["Trveling_hour"]=Data[DurationTime].str[0].str.split("h").str[0]
    Data["Trveling_min"]=Data[DurationTime].str[1].str.split("m").str[0]

In [11]:
convert_travel_time(df,"Duration")

In [12]:
# converting the categorical value to numerical values.
dic={'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4}
df["Total_Stops"]=df["Total_Stops"].map(dic)

In [13]:
df.drop(labels=["Date_of_Journey","Route","Dep_Time","Arrival_Time","Duration"],axis=1,inplace=True)

In [14]:
# removing the duplicates and Trujet data and nan values.
print(f"The shape of data before removing duplicates is : {df.shape}")
df.drop_duplicates(inplace=True)
df.drop(2878,axis=0,inplace=True)
print(f"The shape of data after removing duplicates is : {df.shape}")

The shape of data before removing duplicates is : (10682, 15)
The shape of data after removing duplicates is : (10459, 15)


In [15]:
df["Trveling_min"]=df["Trveling_min"].fillna(0)
df["Total_Stops"]=df["Total_Stops"].fillna(0)

In [16]:
# Converting the object type to Integer type.
df["Dep_hour"] = df["Dep_hour"].astype(int)
df["Dep_min"] = df["Dep_min"].astype(int)
df["Arival_hour"] = df["Arival_hour"].astype(int)
df["Arival_min"] = df["Arival_min"].astype(int)
df["Trveling_hour"] = df["Trveling_hour"].astype(int)
df["Trveling_min"] = df["Trveling_min"].astype(int)
df["Total_Stops"] = df["Total_Stops"].astype(int)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10459 entries, 0 to 10682
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10459 non-null  object
 1   Source           10459 non-null  object
 2   Destination      10459 non-null  object
 3   Total_Stops      10459 non-null  int64 
 4   Additional_Info  10459 non-null  object
 5   Price            10459 non-null  int64 
 6   Day              10459 non-null  int32 
 7   Month            10459 non-null  int32 
 8   Year             10459 non-null  int32 
 9   Dep_hour         10459 non-null  int64 
 10  Dep_min          10459 non-null  int64 
 11  Arival_hour      10459 non-null  int64 
 12  Arival_min       10459 non-null  int64 
 13  Trveling_hour    10459 non-null  int64 
 14  Trveling_min     10459 non-null  int64 
dtypes: int32(3), int64(8), object(4)
memory usage: 1.2+ MB


In [18]:
df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Day,Month,Year,Dep_hour,Dep_min,Arival_hour,Arival_min,Trveling_hour,Trveling_min
0,IndiGo,Banglore,New Delhi,0,No info,3897,24,3,2019,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,2,No info,7662,1,5,2019,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,2,No info,13882,9,6,2019,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,1,No info,6218,12,5,2019,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,1,No info,13302,1,3,2019,16,50,21,35,4,45


In [19]:
# Split the columns to independent and dependent columns.
X = df.drop(labels="Price",axis=1)
Y = df[["Price"]]

In [20]:
# Split the columns to categorical and numerical columns.
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [21]:
from sklearn.preprocessing import OneHotEncoder

In [22]:
categorical_cols

Index(['Airline', 'Source', 'Destination', 'Additional_Info'], dtype='object')

### When we use OneHotEncoder technique scaling is not required for categorical columns.

In [23]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OneHotEncoder(handle_unknown='ignore')),
    ('scaler',StandardScaler(with_mean=False))
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [24]:
# Split the data to training and testing.
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [25]:
X_train.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Day,Month,Year,Dep_hour,Dep_min,Arival_hour,Arival_min,Trveling_hour,Trveling_min
6084,Air India,Banglore,New Delhi,1,No info,9,3,2019,22,0,11,5,13,5
9311,Jet Airways,Banglore,New Delhi,1,No info,6,3,2019,22,50,9,30,10,40
5372,Jet Airways Business,Banglore,New Delhi,1,Business class,1,3,2019,5,45,12,25,6,40
1000,IndiGo,Delhi,Cochin,1,No info,21,3,2019,18,35,1,35,7,0
1937,Jet Airways,Kolkata,Banglore,1,No info,1,5,2019,9,35,14,25,28,50


In [26]:
X_test.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Day,Month,Year,Dep_hour,Dep_min,Arival_hour,Arival_min,Trveling_hour,Trveling_min
6020,Multiple carriers,Delhi,Cochin,1,No info,18,5,2019,9,0,21,0,12,0
9399,Jet Airways,Kolkata,Banglore,1,No info,24,3,2019,8,25,18,10,9,45
8261,Air Asia,Kolkata,Banglore,0,No info,21,4,2019,10,20,12,55,2,35
9831,Jet Airways,Banglore,Delhi,0,No info,6,5,2019,17,45,20,45,3,0
8934,Jet Airways,Banglore,New Delhi,1,No info,18,3,2019,14,5,8,15,18,10


In [27]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [28]:
X_train.head()

,num_pipeline__Total_Stops,num_pipeline__Day,num_pipeline__Month,num_pipeline__Year,num_pipeline__Dep_hour,num_pipeline__Dep_min,num_pipeline__Arival_hour,num_pipeline__Arival_min,num_pipeline__Trveling_hour,num_pipeline__Trveling_min,...,cat_pipeline__Destination_New Delhi,cat_pipeline__Additional_Info_1 Long layover,cat_pipeline__Additional_Info_2 Long layover,cat_pipeline__Additional_Info_Business class,cat_pipeline__Additional_Info_Change airports,cat_pipeline__Additional_Info_In-flight meal not included,cat_pipeline__Additional_Info_No Info,cat_pipeline__Additional_Info_No check-in baggage included,cat_pipeline__Additional_Info_No info,cat_pipeline__Additional_Info_Red-eye flight
0,0.304855,-0.516594,-1.466106,0.0,1.675114,-1.292438,-0.357763,-1.174740,0.359121,-1.383022,...,3.542964,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.418141,0.0
1,0.304855,-0.870801,-1.466106,0.0,1.675114,1.368025,-0.651723,0.327858,-0.000508,0.691127,...,3.542964,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.418141,0.0
2,0.304855,-1.461146,-1.466106,0.0,-1.295824,1.101979,-0.210783,0.027339,-0.480013,0.691127,...,3.542964,0.0,0.0,60.510332,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.304855,0.900234,-1.466106,0.0,0.976070,0.569886,-1.827562,0.628378,-0.360136,-1.679329,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.418141,0.0
4,0.304855,-1.461146,0.252882,0.0,-0.596780,0.569886,0.083177,0.027339,2.157265,1.283740,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.418141,0.0


In [29]:
X_test.head()

,num_pipeline__Total_Stops,num_pipeline__Day,num_pipeline__Month,num_pipeline__Year,num_pipeline__Dep_hour,num_pipeline__Dep_min,num_pipeline__Arival_hour,num_pipeline__Arival_min,num_pipeline__Trveling_hour,num_pipeline__Trveling_min,...,cat_pipeline__Destination_New Delhi,cat_pipeline__Additional_Info_1 Long layover,cat_pipeline__Additional_Info_2 Long layover,cat_pipeline__Additional_Info_Business class,cat_pipeline__Additional_Info_Change airports,cat_pipeline__Additional_Info_In-flight meal not included,cat_pipeline__Additional_Info_No Info,cat_pipeline__Additional_Info_No check-in baggage included,cat_pipeline__Additional_Info_No info,cat_pipeline__Additional_Info_Red-eye flight
0,0.304855,0.546027,0.252882,0.0,-0.596780,-1.292438,1.112037,-1.475259,0.239245,-1.679329,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.418141,0.0
1,0.304855,1.254441,-1.466106,0.0,-0.771541,0.037794,0.671097,-0.874220,-0.120384,0.987433,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.418141,0.0
2,-1.216509,0.900234,-0.606612,0.0,-0.422019,-0.228253,-0.210783,1.830456,-0.959518,0.394820,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.418141,0.0
3,-1.216509,-0.870801,0.252882,0.0,0.801309,1.101979,0.965057,1.229417,-0.839641,-1.679329,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.418141,0.0
4,0.304855,0.546027,-1.466106,0.0,0.277025,-1.026391,-0.798703,-0.573701,0.958503,-1.086715,...,3.542964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.418141,0.0


In [30]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC 

In [46]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet(),
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(),
    'LogisticRegression':LogisticRegression(),
    'LinearSVC':LinearSVC()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 77030654955483.72
MAE: 1398403798812.9946
R2 score -2.6548291670302716e+22




/config/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.133e+08, tolerance: 1.535e+07
  model = cd_fast.enet_coordinate_descent(


Lasso
Model Training Performance
RMSE: 2579.9211508931025
MAE: 1768.701011794417
R2 score 70.22016649333415


Ridge
Model Training Performance
RMSE: 2581.74588397081
MAE: 1769.13702695302
R2 score 70.17802608591717


Elasticnet
Model Training Performance
RMSE: 2791.540497392414
MAE: 1912.8553464205133
R2 score 65.13439026468237


DecisionTree
Model Training Performance
RMSE: 1854.96087975994
MAE: 755.4963352453792
R2 score 84.60502778779227




/config/.local/lib/python3.8/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForest
Model Training Performance
RMSE: 1559.1276985337022
MAE: 683.6644659751939
R2 score 89.12391048337031




/config/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/config/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/config/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example us

LogisticRegression
Model Training Performance
RMSE: 2288.741934157873
MAE: 1071.3581899298918
R2 score 76.56294425622053


LinearSVC
Model Training Performance
RMSE: 2414.4422134396827
MAE: 1057.3696622052262
R2 score 73.91787148194256




/config/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [66]:
r2_list
for i in r2_list:
    print(f"{i} : {round(i*100,2)}") 

-2.6548291670302717e+20 : -2.6548291670302716e+22
0.7022016649333415 : 70.22
0.7017802608591717 : 70.18
0.6513439026468237 : 65.13
0.8460502778779226 : 84.61
0.8912391048337032 : 89.12
0.7656294425622053 : 76.56
0.7391787148194255 : 73.92


In [77]:
a =pd.DataFrame(r2_list,model_list).round(decimals=2)
a.sort_values(by=0)

,0
LinearRegression,-2.654829e+20
Elasticnet,6.500000e-01
Lasso,7.000000e-01
Ridge,7.000000e-01
LinearSVC,7.400000e-01
LogisticRegression,7.700000e-01
DecisionTree,8.500000e-01
RandomForest,8.900000e-01
